In [1]:
import torch
print(torch.cuda.is_available())        #
print(torch.cuda.get_device_name(0))    # 

True
NVIDIA GeForce RTX 3060


In [2]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain.vectorstores import Chroma
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

In [3]:
%pwd

'c:\\d\\generative AI\\Source-Code-Analysis-Generative-AI-LLama2\\research'

In [4]:
!mkdir test_repo

In [5]:
repo_path = "test_repo/"

repo = Repo.clone_from("https://github.com/Vinhnv0901/universal-document-QA_with_Llama2.git", to_path=repo_path)

In [6]:
loader = GenericLoader.from_filesystem(repo_path,
                                        glob = "**/*",
                                       suffixes=[".py"],
                                       parser = LanguageParser(language=Language.PYTHON, parser_threshold=500)
)

In [7]:
documents = loader.load()

In [8]:
documents

[Document(metadata={'source': 'test_repo\\ test.py', 'language': <Language.PYTHON: 'python'>}, page_content=''),
 Document(metadata={'source': 'test_repo\\app.py', 'language': <Language.PYTHON: 'python'>}, page_content='from flask import Flask, render_template, jsonify, request\nfrom src.helper import download_hugging_face_embeddings\n# from langchain_pinecone import PineconeVectorStore\nfrom langchain.vectorstores import Chroma\nfrom langchain_openai import OpenAI\nfrom langchain.chains import create_retrieval_chain\nfrom langchain.chains.combine_documents import create_stuff_documents_chain\nfrom langchain_core.prompts import ChatPromptTemplate\nfrom dotenv import load_dotenv\nfrom src.prompt import *\nimport os\nfrom transformers import pipeline\nimport torch\nfrom huggingface_hub import login\nfrom transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig\nfrom langchain import HuggingFacePipeline\napp = Flask(__name__)\n\nload_dotenv()\ntoken = os.environ["HUGGIN

In [9]:
len(documents)

8

In [10]:
documents[0]

Document(metadata={'source': 'test_repo\\ test.py', 'language': <Language.PYTHON: 'python'>}, page_content='')

In [11]:
documents_splitter = RecursiveCharacterTextSplitter.from_language(language = Language.PYTHON,
                                                             chunk_size = 500,
                                                             chunk_overlap = 20)

In [12]:
texts = documents_splitter.split_documents(documents)

In [13]:
texts

[Document(metadata={'source': 'test_repo\\app.py', 'language': <Language.PYTHON: 'python'>}, page_content='from flask import Flask, render_template, jsonify, request\nfrom src.helper import download_hugging_face_embeddings\n# from langchain_pinecone import PineconeVectorStore\nfrom langchain.vectorstores import Chroma\nfrom langchain_openai import OpenAI\nfrom langchain.chains import create_retrieval_chain\nfrom langchain.chains.combine_documents import create_stuff_documents_chain\nfrom langchain_core.prompts import ChatPromptTemplate\nfrom dotenv import load_dotenv\nfrom src.prompt import *\nimport os'),
 Document(metadata={'source': 'test_repo\\app.py', 'language': <Language.PYTHON: 'python'>}, page_content='import os\nfrom transformers import pipeline\nimport torch\nfrom huggingface_hub import login\nfrom transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig\nfrom langchain import HuggingFacePipeline\napp = Flask(__name__)'),
 Document(metadata={'source': 'tes

In [14]:
len(texts)

14

In [15]:
from huggingface_hub import login
import os
from dotenv import load_dotenv

# Load token từ .env
load_dotenv()
token = os.environ["HUGGINGFACEHUB_API_TOKEN"]

# Login trực tiếp bằng code
login(token=token)

print("✅ Đã login thành công!")

c:\d\generative AI\Source-Code-Analysis-Generative-AI-LLama2\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Đã login thành công!


In [16]:
from langchain.embeddings import HuggingFaceEmbeddings
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [17]:
embeddings = download_hugging_face_embeddings()

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_15812\3158813755.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [18]:
vectordb = Chroma.from_documents(texts, embedding=embeddings, persist_directory='./db')

In [19]:
vectordb.persist()

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_15812\3711397106.py:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [20]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model = "meta-llama/Llama-2-7b-chat-hf"
bnb_config = BitsAndBytesConfig(load_in_4bit=True)

tokenizer = AutoTokenizer.from_pretrained(model, token=token)
model = AutoModelForCausalLM.from_pretrained(
    model,
    device_map="auto",
    quantization_config=bnb_config,
    token=token
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:24<00:00, 12.32s/it]


In [21]:
from transformers import pipeline
pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

Device set to use cuda:0


In [22]:
from langchain import HuggingFacePipeline
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_15812\2412414649.py:2: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})


In [23]:
memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_15812\2101274949.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)


In [24]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k":8}), memory=memory)

In [25]:
question = "what is download_hugging_face_embeddings funtion?"

In [26]:
result = qa(question)
print(result['answer'])

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_15812\79176006.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa(question)


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks



#Download the Embeddings from HuggingFace 
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')  #this model return 384 dimensions
    return embeddings

from src.helper import load_pdf_file, text_split, download_hugging_face_embeddings
from langchain.vectorstores import Chroma






extracted_data=load_pdf_file(data='data/')
text_chunks=text_split(extracted_data)
embeddings = download_hugging_face_embeddings()
persist_directory = 'db'


vectordb = Chroma.from_documents(documents=text_chunks,
                                 embedding=embeddings,
   

In [27]:
question = "what is load_pdf_file funtion?"
result = qa(question)
print(result['answer'])

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks



#Download the Embeddings from HuggingFace 
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')  #this model return 384 dimensions
    return embeddings

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

def index():
    return render_template('chat.html')


@app.route("/get", methods=["GET